<a href="https://colab.research.google.com/github/bonofaber/Auto-GPT-making/blob/master/Titanic_XGBoost__RandomizedSearchCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier


In [ ]:
# 구글 드라이브 마운트  => 인증 및 액세스 허
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 2. 데이터 불러오기 - kaggle 사용
#train = pd.read_csv("/kaggle/input/titanic/train.csv")
#test = pd.read_csv("/kaggle/input/titanic/test.csv")

In [ ]:
# 경로를 실제 파일이 있는 구글 드라이브 경로로 수정
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Titanic/train.csv")
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Titanic/test.csv")

print(train.head())
print(test.head())

In [ ]:
train.info()          # 데이터 타입, 결측치 등
train.describe()      # 통계 요약
#train['Sex'].value_counts()  # 범주형 분석
train.isnull().sum()  # 결측치 개수

In [ ]:
# 결측치 시각화
import missingno as msno # null data 쉽게 보여주는 library
msno.matrix(train, figsize=(8, 3))
msno.matrix(test, figsize=(8, 3))

In [ ]:
# 3. 기존 전처리
"""
train['Age'] = train['Age'].fillna(train['Age'].median())
train['Embarked'] = train['Embarked'].fillna('S')
test['Age'] = test['Age'].fillna(test['Age'].median())
test['Fare'] = test['Fare'].fillna(test['Fare'].median())

train['Sex'] = train['Sex'].map({'male': 0, 'female': 1})
test['Sex'] = test['Sex'].map({'male': 0, 'female': 1})
train['Embarked'] = train['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
test['Embarked'] = test['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
"""

In [ ]:
#3-1 전처리 수정 - 머신러닝 모델 활용 (K-Nearest Neighbors) 🤖
""" scikit-learn의 KNNImputer 사용
 다른 모든 수치 특성을 고려하여, 현재 행과 가장 유사한 'K'개의 이웃 데이터를 찾아 그 값들의 평균으로 결측치를 채움
 """

import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder


# 2. 숫자형 특성 선택 및 스케일링
impute_features = ['Age', 'Fare', 'Pclass', 'SibSp', 'Parch']

# 스케일링 적용
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train[impute_features])
test_scaled = scaler.transform(test[impute_features])

# KNN Imputer 적용
imputer = KNNImputer(n_neighbors=5)
train[impute_features] = pd.DataFrame(imputer.fit_transform(train_scaled), columns=impute_features)
test[impute_features] = pd.DataFrame(imputer.transform(test_scaled), columns=impute_features)

# 스케일링 복원 (필요한 경우)
train[impute_features] = scaler.inverse_transform(train[impute_features])
test[impute_features] = scaler.inverse_transform(test[impute_features])

# 3. Embarked 결측치 처리 (최빈값 사용)
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].mode()[0])
test['Embarked'] = test['Embarked'].fillna(train['Embarked'].mode()[0])

# 4. 범주형 변수 인코딩
label_encoder_sex = LabelEncoder()
train['Sex'] = label_encoder_sex.fit_transform(train['Sex'])
test['Sex'] = label_encoder_sex.transform(test['Sex'])

label_encoder_embarked = LabelEncoder()
train['Embarked'] = label_encoder_embarked.fit_transform(train['Embarked'])
test['Embarked'] = label_encoder_embarked.transform(test['Embarked'])

In [ ]:
# 결측치 시각화
import missingno as msno # null data 쉽게 보여주는 library
msno.matrix(train, figsize=(8, 3))
msno.matrix(test, figsize=(8, 3))

In [ ]:

# 4. 특성 선택 및 데이터 분리
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = train[features]
y = train['Survived']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 비교) 모델 생성 및 학습
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)  # 경고 방지용 옵션 포함
xgb_model.fit(X_train, y_train)

# 비교) 검증 데이터셋 예측
y_pred = xgb_model.predict(X_val)

# 비교) 성능 평가
print(classification_report(y_val, y_pred))
print("Accuracy:", accuracy_score(y_val, y_pred))

In [ ]:

# 5. XGBoost 하이퍼파라미터 범위 지정
xgb_params = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [4, 6, 8, 10, 20, 30, 40],
    'learning_rate': [0.1, 0.2, 0.3],
    'min_child_weight': [2, 4, 6, 8, 10],
    'subsample': [0.2, 0.3, 0.4, 0.5, 0.6]
}


In [ ]:
# 6. RandomizedSearchCV 객체 생성
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')  # 경고 방지를 위해 옵션 추가
xgb_rs_cv = RandomizedSearchCV(
    xgb,
    param_distributions=xgb_params,
    cv=5,               # 5-Fold 교차검증
    n_iter=20,          # 20회 조합 시도
    n_jobs=-1,          # 모든 CPU 사용
    random_state=42,
    verbose=1
)


In [ ]:
# 7. 튜닝 수행
xgb_rs_cv.fit(X_train, y_train)

In [ ]:
# 8. 최적 파라미터 확인

""" 당초 파라미터
최적 하이퍼파라미터: {'subsample': 0.2, 'n_estimators': 200, 'min_child_weight': 2, 'max_depth': 6, 'learning_rate': 0.1}
최고 정확도: 0.8272727272727274
"""

print(f"최적 하이퍼파라미터: {xgb_rs_cv.best_params_}")
print(f"최고 정확도: {xgb_rs_cv.best_score_}")


In [ ]:
# 9. 검증셋 성능 확인
""" 당초 결과
              precision    recall  f1-score   support

           0       0.85      0.87      0.86       105
           1       0.81      0.78      0.79        74

    accuracy                           0.83       179
   macro avg       0.83      0.83      0.83       179
weighted avg       0.83      0.83      0.83       179

Accuracy: 0.8324022346368715
"""

best_model = xgb_rs_cv.best_estimator_
y_pred = best_model.predict(X_val)
print(classification_report(y_val, y_pred))
print("Accuracy:", accuracy_score(y_val, y_pred))


In [ ]:
# 10. 테스트셋 예측 및 캐글 제출 csv 생성
X_test = test[features]
preds = best_model.predict(X_test)

submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': preds
})

#submission.to_csv('submission_xgb.csv', index=False)
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/Titanic/submission_xgb.csv', index=False)
print("파일이 '/content/drive/MyDrive/Colab Notebooks/Titanic/' 폴더에 저장되었습니다.")